# Analysis of NCAA Basketball Data
In this notebook, I analyze a variety of information about NCAA basketball. I use a large database hosted on Google Bigquery, which can be found at https://www.kaggle.com/ncaa/ncaa-basketball.


The database contains the following tables:
- mascots
- team_colors
- mbb_teams
- mbb_historical_teams_seasons - historical season data (1894/1895-present)
- mbb_historical_teams_games - final scores, one entry per team per game (1996/1997-present)
- mbb_historical_tournament_games - historical data about tournament games (1984/1985-present)
- mbb_games_sr - team level box scores (2013/2014-2017/2018)
- mbb_pbp_sr - play by play information about games (2013/2014-present)
- mbb_players_games_sr - player level box scores (2013/2014-2017/2018)
- mbb_teams_games_sr - team level box scores (2013/2014-2017/2018)

Because I attended Boston College and Wisconsin, and Wisconsin has the much better basketball team, I will mainly attempt to get a sense of Wisconsin's program and its performance over the years. Based on the database tables, it looks like I may be able to answer the following questions:
- What is Wisconin's mascot?
- What is Wisconsin's team color?
- Where does Wisconsin play home games?
- Where has Wisconsin played the most games?
- Home vs. away performance
- Highest scoring games
- Tournament performance
- Chokes vs. Upsets

In [1]:
# Import packages
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
from google.cloud import bigquery
%matplotlib inline

# Accessing Kaggle data files
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [2]:
# Retrieve NCAA data
client = bigquery.Client()
dataset_ref = client.dataset("ncaa_basketball", project="bigquery-public-data")
dataset = client.get_dataset(dataset_ref)

Using Kaggle's public dataset BigQuery integration.


In [3]:
# Make sure we have the correct database
tables = list(client.list_tables(dataset))
for table in tables:  
    print(table.table_id)

mascots
mbb_games_sr
mbb_historical_teams_games
mbb_historical_teams_seasons
mbb_historical_tournament_games
mbb_pbp_sr
mbb_players_games_sr
mbb_teams
mbb_teams_games_sr
team_colors


# General Information
To start off, we'll take a peek at some of the general information held in the following tables:
- mascots
- team_colors
- mbb_teams

# Exploring Mascots

In [4]:
# Select the mascot table
table_ref = dataset_ref.table("mascots")
table = client.get_table(table_ref)

In [5]:
# Check layout of mascots table
table.schema

[SchemaField('id', 'STRING', 'NULLABLE', 'University unique ID from Sportradar', (), None),
 SchemaField('market', 'STRING', 'NULLABLE', 'The university to which the mascot belongs', (), None),
 SchemaField('name', 'STRING', 'NULLABLE', 'The name of the university’s team', (), None),
 SchemaField('mascot', 'STRING', 'NULLABLE', "The name of the university's mascot", (), None),
 SchemaField('mascot_name', 'STRING', 'NULLABLE', "The proper name of the university's mascot, if available (e.g. a character)", (), None),
 SchemaField('mascot_common_name', 'STRING', 'NULLABLE', 'The type of being or creature that the mascot embodies', (), None),
 SchemaField('tax_subspecies', 'STRING', 'NULLABLE', 'The subspecies to which the mascot belongs', (), None),
 SchemaField('tax_species', 'STRING', 'NULLABLE', 'The species to which the mascot belongs', (), None),
 SchemaField('tax_genus', 'STRING', 'NULLABLE', 'The genus to which the mascot belongs', (), None),
 SchemaField('tax_family', 'STRING', 'NU

In [6]:
# Some example records
client.list_rows(table, max_results=5).to_dataframe()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Cannot use bqstorage_client if max_results is set, reverting to fetching data with the tabledata.list endpoint.
  


,id,market,name,mascot,mascot_name,mascot_common_name,tax_subspecies,tax_species,tax_genus,tax_family,tax_order,tax_class,tax_phylum,tax_kingdom,tax_domain,non_tax_type
0,2959bd24-7007-41ae-a3a3-abdf26888cfc,Tulsa,Golden Hurricane,Hurricane,Captain Cane,None,None,None,None,None,None,None,None,None,None,Weather
1,ad4bc983-8d2e-4e6f-a8f9-80840a786c64,Arizona State,Sun Devils,Devil,Sparky,None,None,None,None,None,None,None,None,None,None,Devils
2,f2d01b77-0f5d-4574-9e49-2a3eaf822e44,Drexel,Dragons,Dragon,Mario the Magnificent,None,None,None,None,None,None,None,None,None,None,Dragons
3,b47d10b8-a2a5-47df-a2f9-7bd0b9d51beb,Bradley,Braves,Gargoyle,Kaboom!,None,None,None,None,None,None,None,None,None,None,None
4,0113eea0-c943-4fff-9780-ae0fb099e7ef,Canisius,Golden Griffins,Griffin,Petey,None,None,None,None,None,None,None,None,None,None,Griffins


In [7]:
# As a sanity check, let's see Wisconsin's mascot
query = """
        SELECT *
        FROM `bigquery-public-data.ncaa_basketball.mascots`
        WHERE market = 'Wisconsin'
        """

client = bigquery.Client()
query_job = client.query(query)
wisc = query_job.to_dataframe()
wisc

Using Kaggle's public dataset BigQuery integration.


/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


,id,market,name,mascot,mascot_name,mascot_common_name,tax_subspecies,tax_species,tax_genus,tax_family,tax_order,tax_class,tax_phylum,tax_kingdom,tax_domain,non_tax_type
0,c7569eae-5b93-4197-b204-6f3a62146b25,Wisconsin,Badgers,Badger,Bucky,Badger,None,None,None,Mustelidae,Carnivora,Mammalia,Chordata,Animalia,Eukaryota,None


![](https://cdn.wrn.com/wp-content/uploads/2012/09/BuckyBadger.jpg)

In [8]:
# Are there any other badger mascots?
query = """
        SELECT market, name, mascot, mascot_name
        FROM `bigquery-public-data.ncaa_basketball.mascots`
        WHERE mascot = 'Badger'
        """

client = bigquery.Client()
query_job = client.query(query)
badgers = query_job.to_dataframe()
badgers

Using Kaggle's public dataset BigQuery integration.


/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


,market,name,mascot,mascot_name
0,Wisconsin,Badgers,Badger,Bucky


In [9]:
# Are there any other mascots named Bucky?
query = """
        SELECT market, name, mascot, mascot_name
        FROM `bigquery-public-data.ncaa_basketball.mascots`
        WHERE mascot_name = 'Bucky'
        """

client = bigquery.Client()
query_job = client.query(query)
buckys = query_job.to_dataframe()
buckys

Using Kaggle's public dataset BigQuery integration.


/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


,market,name,mascot,mascot_name
0,East Tennessee State,Buccaneers,Pirate,Bucky
1,Charleston Southern,Buccaneers,Pirate,Bucky
2,Wisconsin,Badgers,Badger,Bucky
3,Bucknell,Bison,Bison,Bucky
4,Texas Rio Grande Valley,Vaqueros,Horse,Bucky
5,Santa Clara,Broncos,Bronco,Bucky


In [10]:
# What is the most common mascot name?
query = """
        SELECT mascot_name, count(mascot_name) as total
        FROM `bigquery-public-data.ncaa_basketball.mascots`
        GROUP BY mascot_name
        ORDER BY total DESC
        LIMIT 10
        """

client = bigquery.Client()
query_job = client.query(query)
name_count = query_job.to_dataframe()
name_count

Using Kaggle's public dataset BigQuery integration.


/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


,mascot_name,total
0,Bucky,6
1,Rowdy,5
2,Willie,4
3,Big Red,3
4,Sparky,3
5,Benny,3
6,Swoop,3
7,Sparty,2
8,Sammy,2
9,Pistol Pete,2


In [11]:
# What is the most common mascot
query = """
        SELECT mascot, count(mascot) as total
        FROM `bigquery-public-data.ncaa_basketball.mascots`
        GROUP BY mascot
        ORDER BY total DESC
        LIMIT 10
        """

client = bigquery.Client()
query_job = client.query(query)
masc_count = query_job.to_dataframe()
masc_count

Using Kaggle's public dataset BigQuery integration.


/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


,mascot,total
0,Bulldog,19
1,Tigers,13
2,Eagle,12
3,Wildcat,10
4,Brown Bear,9
5,Hawk,9
6,Panther,8
7,Wolf,8
8,Cougar,7
9,Lion,7


In [12]:
# What is the most common non-animal mascot
query = """
        SELECT non_tax_type, count(non_tax_type) as total
        FROM `bigquery-public-data.ncaa_basketball.mascots`
        GROUP BY non_tax_type
        ORDER BY total DESC
        LIMIT 10
        """

client = bigquery.Client()
query_job = client.query(query)
masc_count = query_job.to_dataframe()
masc_count

Using Kaggle's public dataset BigQuery integration.


/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


,non_tax_type,total
0,Warriors,10
1,Devils,7
2,Seafarers,7
3,Aristocrats,6
4,Explorers,6
5,Cowboys,4
6,Mythical Creatures,4
7,Weather,3
8,Dragons,3
9,Clergy,2


# Team Colors

In [13]:
# Select the team colors table
table_ref = dataset_ref.table("team_colors")
table = client.get_table(table_ref)

In [14]:
# Check layout of colors table
table.schema

[SchemaField('market', 'STRING', 'NULLABLE', None, (), None),
 SchemaField('id', 'STRING', 'NULLABLE', None, (), None),
 SchemaField('code_ncaa', 'INTEGER', 'NULLABLE', None, (), None),
 SchemaField('color', 'STRING', 'NULLABLE', None, (), None)]

In [15]:
# Some example records
client.list_rows(table, max_results=5).to_dataframe()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Cannot use bqstorage_client if max_results is set, reverting to fetching data with the tabledata.list endpoint.
  


,market,id,code_ncaa,color
0,Milwaukee,5d77800f-1ae6-4b66-8e97-b0dbb8bbc717,797,#000000
1,Colorado,9fccbf28-2858-4263-821c-fdefb3c7efa3,157,#000000
2,Northeastern,93df9b18-e9fc-42a7-bb45-a736c203b4dc,500,#000000
3,Georgia Southern,6b955b96-b736-475e-bffd-e4acf2054169,253,#000066
4,Richmond,9b66e1e0-aace-4671-9be2-54c8acf5ecfc,575,#000066


In [16]:
# As another sanity check, let's see Wisconsin's color
query = """
        SELECT *
        FROM `bigquery-public-data.ncaa_basketball.team_colors`
        WHERE market = 'Wisconsin'
        """

client = bigquery.Client()
query_job = client.query(query)
wisc = query_job.to_dataframe()
wisc

Using Kaggle's public dataset BigQuery integration.


/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


,market,id,code_ncaa,color
0,Wisconsin,c7569eae-5b93-4197-b204-6f3a62146b25,796,#be0f34


In [17]:
# Wisconsin's color is be0f34, which has relatively high red intensity and low green/blue intensities
# However, are there any teams with maximum red intensity as a color?

In [18]:
# Query teams with higher red intensity than Wisconsin
query = """
        SELECT *
        FROM `bigquery-public-data.ncaa_basketball.team_colors`
        WHERE color LIKE "#FF%" OR color LIKE "#ff%"
        """

client = bigquery.Client()
query_job = client.query(query)
wisc = query_job.to_dataframe()
wisc

Using Kaggle's public dataset BigQuery integration.


/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


,market,id,code_ncaa,color
0,Oklahoma State,8ab00d43-840a-4c96-bdee-bf88fa6e3f11,521,#FF6600
1,South Dakota,c946b7ac-5159-4817-93c4-fc9495c6425b,650,#ff2310
2,Syracuse,8cd24de1-2be8-4b77-bcfa-47be0495a5f4,688,#ff5113
3,Pacific,c9570e17-c4fd-41e6-908a-1a083941319d,534,#ff6900
4,Tennessee-Martin,9febd5fa-982f-4f40-ad1d-8e49be96cf4f,695,#ff6900
5,Idaho State,6a67ba19-56a8-4dd8-a5ae-9e9f2523c274,294,#ff7800
6,Northern Colorado,7e42fa84-68cd-47a6-b49a-18b5414d8084,502,#ffb500
7,North Carolina A&T,53aaac6a-796e-41a6-a04e-6ce74c5717f8,488,#ffb82b
8,Morehead State,b7747a03-c85b-4de6-94bf-1b5b68553248,444,#ffc300


# Wisconsin Team Information

In [19]:
# Select the team information table
table_ref = dataset_ref.table("mbb_teams")
table = client.get_table(table_ref)

In [20]:
# Check layout of team information table
table.schema

[SchemaField('market', 'STRING', 'NULLABLE', None, (), None),
 SchemaField('alias', 'STRING', 'NULLABLE', None, (), None),
 SchemaField('name', 'STRING', 'NULLABLE', None, (), None),
 SchemaField('id', 'STRING', 'NULLABLE', None, (), None),
 SchemaField('code_ncaa', 'INTEGER', 'NULLABLE', None, (), None),
 SchemaField('kaggle_team_id', 'INTEGER', 'NULLABLE', None, (), None),
 SchemaField('school_ncaa', 'STRING', 'NULLABLE', None, (), None),
 SchemaField('turner_name', 'STRING', 'NULLABLE', None, (), None),
 SchemaField('league_name', 'STRING', 'NULLABLE', None, (), None),
 SchemaField('league_alias', 'STRING', 'NULLABLE', None, (), None),
 SchemaField('league_id', 'STRING', 'NULLABLE', None, (), None),
 SchemaField('conf_name', 'STRING', 'NULLABLE', None, (), None),
 SchemaField('conf_alias', 'STRING', 'NULLABLE', None, (), None),
 SchemaField('conf_id', 'STRING', 'NULLABLE', None, (), None),
 SchemaField('division_name', 'STRING', 'NULLABLE', None, (), None),
 SchemaField('division_al

In [21]:
# Some example records
client.list_rows(table, max_results=5).to_dataframe()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Cannot use bqstorage_client if max_results is set, reverting to fetching data with the tabledata.list endpoint.
  


,market,alias,name,id,code_ncaa,kaggle_team_id,school_ncaa,turner_name,league_name,league_alias,...,venue_city,venue_state,venue_address,venue_zip,venue_country,venue_name,venue_capacity,logo_large,logo_medium,logo_small
0,Princeton,PRIN,Tigers,fe406882-9f22-495e-9df6-ef357a6803c6,554,1343,Princeton,Princeton University,NCAA MEN,NCAAM,...,Princeton,NJ,Princeton University,8542,USA,Jadwin Gymnasium,6854,https://www.ncaa.com/sites/default/files/image...,https://www.ncaa.com/sites/default/files/image...,https://www.ncaa.com/sites/default/files/image...
1,Yale,YALE,Bulldogs,ca478771-aa3d-4231-81e0-b70f519134fb,813,1463,Yale,Yale University,NCAA MEN,NCAAM,...,New Haven,CT,70 Tower Pkwy,6511,USA,John J. Lee Amphitheater,2532,https://www.ncaa.com/sites/default/files/image...,https://www.ncaa.com/sites/default/files/image...,https://www.ncaa.com/sites/default/files/image...
2,Harvard,HARV,Crimson,5c7bf63f-bc39-43c5-9907-73b50b7a6b34,275,1217,Harvard,Harvard University,NCAA MEN,NCAAM,...,Allston,MA,Soldiers Field Road,2163,USA,Lavietes Pavilion,2195,https://www.ncaa.com/sites/default/files/image...,https://www.ncaa.com/sites/default/files/image...,https://www.ncaa.com/sites/default/files/image...
3,Dartmouth,DART,Big Green,d60357bd-1205-42e9-9092-d986a2843a34,172,1171,Dartmouth,Dartmouth College,NCAA MEN,NCAAM,...,Hanover,NH,6 South Park Street,03755,USA,Edward Leede Arena,2100,https://www.ncaa.com/sites/default/files/image...,https://www.ncaa.com/sites/default/files/image...,https://www.ncaa.com/sites/default/files/image...
4,Cornell,COR,Big Red,88ff8c00-958e-4ccf-a21d-77fab9e93692,167,1165,Cornell,Cornell University,NCAA MEN,NCAAM,...,Ithaca,NY,Bartels Hall,14853,USA,Newman Arena,4473,https://www.ncaa.com/sites/default/files/image...,https://www.ncaa.com/sites/default/files/image...,https://www.ncaa.com/sites/default/files/image...


In [22]:
# Query Wisconsin's team information
query = """
        SELECT 
            market,
            league_name,
            conf_name,
            division_name,
            venue_name,
            venue_capacity
        FROM `bigquery-public-data.ncaa_basketball.mbb_teams`
        WHERE market = "Wisconsin"
        """

client = bigquery.Client()
query_job = client.query(query)
wisc = query_job.to_dataframe()
wisc

Using Kaggle's public dataset BigQuery integration.


/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


,market,league_name,conf_name,division_name,venue_name,venue_capacity
0,Wisconsin,NCAA MEN,Big Ten,NCAA Division I,Kohl Center,17230


In [23]:
# So, we can see that Wisconin is a NCAA Men's Division I team playing in the Big Ten.
# They play at the Kohl Center, which has a capacity of 17,230.
# Let's check out some other Wisconsin and Big Ten teams.

In [24]:
# Query other teams in Wisconsin
query = """
        SELECT 
            market,
            league_name,
            conf_name,
            division_name,
            venue_name,
            venue_capacity
        FROM `bigquery-public-data.ncaa_basketball.mbb_teams`
        WHERE venue_state = "WI"
        ORDER BY venue_capacity DESC
        """

client = bigquery.Client()
query_job = client.query(query)
wisc = query_job.to_dataframe()
wisc

Using Kaggle's public dataset BigQuery integration.


/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


,market,league_name,conf_name,division_name,venue_name,venue_capacity
0,Marquette,NCAA MEN,Big East,NCAA Division I,BMO Harris Bradley Center,18717
1,Wisconsin,NCAA MEN,Big Ten,NCAA Division I,Kohl Center,17230
2,Milwaukee,NCAA MEN,Horizon,NCAA Division I,UW-Milwaukee Panther Arena,10783
3,Green Bay,NCAA MEN,Horizon,NCAA Division I,Resch Center,9729


In [25]:
# There are three other D1 basketball programs in Wisconsin, but none plays in the Big Ten.
# Marquette has the largest venue in Wisconsin.

In [26]:
# Query other teams in the Big Ten
query = """
        SELECT 
            market,
            venue_name,
            venue_capacity
        FROM `bigquery-public-data.ncaa_basketball.mbb_teams`
        WHERE conf_name = "Big Ten"
        ORDER BY venue_capacity DESC
        """

client = bigquery.Client()
query_job = client.query(query)
big_ten = query_job.to_dataframe()
big_ten

Using Kaggle's public dataset BigQuery integration.


/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


,market,venue_name,venue_capacity
0,Ohio State,Value City Arena at the Jerome Schottenstein C...,18809
1,Maryland,Xfinity Center,17950
2,Northwestern,Allstate Arena,17500
3,Indiana,Assembly Hall,17472
4,Wisconsin,Kohl Center,17230
5,Michigan State,Jack Breslin Students Events Center,16280
6,Illinois,State Farm Center,15500
7,Iowa,Carver-Hawkeye Arena,15400
8,Penn State,Bryce Jordan Center,15261
9,Nebraska,Pinnacle Bank Arena,15147


In [27]:
# Much to my surprise, the Big Ten conference actually has 14 teams.
# Wisconsin has the fifth largest venue in the Big Ten.